# Analyzing Music Sales

In this project we will use the fictional Itunes-like musical sales database "Chinook" to practice SQL queries related mainly to user preferences. The main focus of the second half of the project will be a detailed analysis of whether individual songs or whole albums should be sold.

In [1]:
#Importing database
import sqlalchemy
sqlalchemy.create_engine('sqlite:///factbook.db')
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

In [46]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type in ('table')

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The database contains information on all songs sold in our fictional music store and all transactions completed by customers.


# Task 1: Genre recommendations for USA

Our company is considering adding two of the four artists to their US store and wants to know: which of hip-hop, punk, pop, or blues will sell the best in the US?

To have a more comprehensive picture of user tastes, we'll rank the genres by their popularity in the US. Then we'll see where these 4 fit in those rankings.

Let's start by looking at some of the available genres.

In [3]:
%%sql
SELECT *
FROM genre
LIMIT 10

 * sqlite:///chinook.db
Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll
6,Blues
7,Latin
8,Reggae
9,Pop
10,Soundtrack


In [4]:
%%sql
SELECT COUNT(*)
FROM genre

 * sqlite:///chinook.db
Done.


COUNT(*)
25


So there are 25 genres. 

Next we need to decide how to measure popularity. Since are greedy capitalists, we could look at total revenue or 
total number of songs sold. In this scenario, as each song is the same price, these are equivalent, so we will focus on revenue.

Below, we generate a table ranking genres by their total revenue and percent of total revenue.

In [5]:
%%sql
WITH track_genre AS
        (
        SELECT  t.track_id AS track_id,
                t.genre_id AS genre_id, 
                g.name AS genre_name
        FROM track t
        LEFT JOIN genre g ON g.genre_id = t.genre_id),
    invoice_line_genre_country AS
        (
        SELECT il.*, t.genre_name, t.genre_id, i.billing_country
        FROM invoice_line il
        LEFT JOIN invoice i ON i.invoice_id = il.invoice_id
        LEFT JOIN track_genre t ON t.track_id = il.track_id)
        
SELECT  genre_name, 
        ROUND(SUM(unit_price * quantity),2) AS total_US_revenue,
        ROUND(SUM(unit_price * quantity) / (SELECT SUM(unit_price * quantity) FROM invoice_line_genre_country
                                      WHERE billing_country = 'USA'),2) AS percent_US_revenue
FROM invoice_line_genre_country
WHERE billing_country = 'USA'
GROUP BY genre_name
ORDER BY total_US_revenue DESC

 * sqlite:///chinook.db
Done.


genre_name,total_US_revenue,percent_US_revenue
Rock,555.39,0.53
Alternative & Punk,128.7,0.12
Metal,122.76,0.12
R&B/Soul,52.47,0.05
Blues,35.64,0.03
Alternative,34.65,0.03
Pop,21.78,0.02
Latin,21.78,0.02
Hip Hop/Rap,19.8,0.02
Jazz,13.86,0.01


## Analysis

Based on the table above, rock and alternative/punk were most popular based on total sales. In the four categories we're exploring, the order would be:
- Alternative / Punk (12% of sales)
- Blues (3%)
- Pop (2%)
- Hip Hop / Rap (2%)

There is some uncertainty in this, as the 'alternative/punk' category may be more popular because of alternative rather than punk. However, there is a separate 'alternative' category, so we may be reasonably confident it's our best choice. It has about 4 times the sales of the next most popular option, Blues.

So definitely choose the punk album. For the other three, it's much closer.

# Task 2: Sales Support Agent Performance

Next we can check how well each sales support agent is performing. We will first list the employees and their characteristics, then determine how much each agent has sold.

In [6]:
%%sql

SELECT employee_id, first_name, last_name, title, birthdate, hire_date, country
FROM employee
WHERE title = 'Sales Support Agent'

 * sqlite:///chinook.db
Done.


employee_id,first_name,last_name,title,birthdate,hire_date,country
3,Jane,Peacock,Sales Support Agent,1973-08-29 00:00:00,2017-04-01 00:00:00,Canada
4,Margaret,Park,Sales Support Agent,1947-09-19 00:00:00,2017-05-03 00:00:00,Canada
5,Steve,Johnson,Sales Support Agent,1965-03-03 00:00:00,2017-10-17 00:00:00,Canada


In [7]:
%%sql 
WITH rep_totals AS
    (
    SELECT support_rep_id, ROUND(SUM(total),2) AS total_sold
    FROM invoice i
    LEFT JOIN customer c ON c.customer_id = i.customer_id
    GROUP BY support_rep_id
    LIMIT 10
    )
SELECT  first_name || " " || last_name AS employee_name,
        total_sold,
        ROUND(total_sold / (SELECT SUM(total_sold) FROM rep_totals),2) AS percent_sales
FROM employee e
INNER JOIN rep_totals t ON t.support_rep_id = e.employee_id
ORDER BY total_sold DESC

 * sqlite:///chinook.db
Done.


employee_name,total_sold,percent_sales
Jane Peacock,1731.51,0.37
Margaret Park,1584.0,0.34
Steve Johnson,1393.92,0.3


## Analysis

Jane has the highest percent of sales (37%), followed closely by Margaret and Steve.

Jane and Margaret have both been working at the company for at least 5 months more than Steve, so it makes sense that they would have more sales. Depending on when this data was taken, Steve may in fact have relatively more sales for the time spent working there.

# Task 3: Sales by Country

We would like to analyze the store's relative performance in different countries. For each country, we'll find:
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

The table below lists the top ten countries by total number of distinct customers.

In [33]:
%%sql

SELECT country, COUNT(i.customer_id) as total_purchases, COUNT(DISTINCT(i.customer_id)) as total_customers
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
GROUP BY country
ORDER BY 3 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


country,total_purchases,total_customers
USA,131,13
Canada,76,8
France,50,5
Brazil,61,5
Germany,41,4
United Kingdom,28,3
Portugal,29,2
India,21,2
Czech Republic,30,2
Sweden,10,1


It appears from the difference in "total purchases" and "total customers" that there are repeat customers. Let's verify this by checking all of India's purchases:

In [10]:
%%sql
SELECT invoice_id, i.customer_id
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
WHERE country = 'India'

 * sqlite:///chinook.db
Done.


invoice_id,customer_id
33,58
82,58
100,58
140,58
146,58
246,58
285,58
294,58
320,58
338,58


Confirmed - there are two distinct customers with multiple purchases totaling 21 as in the previous table. So we'll count by distinct customer IDs when calculating our totals.

The table below shows total sales (\\$), average sales per customer (\\$), and average sales per order (number of songs) for each country.

In [11]:
%%sql

SELECT  country, 
        COUNT(DISTINCT(i.customer_id)) AS total_customers,
        ROUND(SUM(total),2) AS total_sales,
        ROUND(SUM(total) / COUNT(DISTINCT(i.customer_id)),2) AS avg_sales_per_customer,
        ROUND(SUM(total) / COUNT(i.invoice_id),1) AS avg_sales_per_order
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
GROUP BY country
ORDER BY total_customers DESC

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_sales_per_order
USA,13,1040.49,80.04,7.9
Canada,8,535.59,66.95,7.0
France,5,389.07,77.81,7.8
Brazil,5,427.68,85.54,7.0
Germany,4,334.62,83.66,8.2
United Kingdom,3,245.52,81.84,8.8
Portugal,2,185.13,92.57,6.4
India,2,183.15,91.58,8.7
Czech Republic,2,273.24,136.62,9.1
Sweden,1,75.24,75.24,7.5


Since there are many countries with just one sale, to simplify this chart we'll group them into one "Other" category. We do this by creating the following view:

In [38]:
%%sql
CREATE VIEW country_sales AS
    
    WITH countries_simplified AS
        (
        SELECT  country,
            CASE
            WHEN COUNT(DISTINCT(i.customer_id)) = 1 THEN 'Other'
            ELSE country END AS country_simplified
        FROM invoice i
        LEFT JOIN customer c ON c.customer_id = i.customer_id
        GROUP BY country
        )
    
SELECT  cs.country_simplified AS country,
        COUNT(DISTINCT(i.customer_id)) AS total_customers,
        ROUND(SUM(total),2) AS total_sales,
        ROUND(SUM(total) / COUNT(DISTINCT(i.customer_id)),2) AS avg_sales_per_customer,
        ROUND(SUM(total) / COUNT(i.invoice_id),1) AS avg_sales_per_order
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
LEFT JOIN countries_simplified cs ON cs.country = c.country
GROUP BY 1
ORDER BY total_customers DESC;

 * sqlite:///chinook.db
Done.


[]

Now we sort this view so that the "Other" category appears at the end, and countries are otherwise sorted by total number of customers.

In [13]:
%%sql
WITH country_sales_sorted AS
    (
    SELECT  cs.*,
            CASE 
            WHEN cs.country = 'Other' THEN 1
            ELSE 0 
            END AS sort
    FROM country_sales cs
    )
SELECT  country, 
        total_customers, 
        total_sales, 
        avg_sales_per_customer, 
        avg_sales_per_order
FROM country_sales_sorted
ORDER BY sort, 2 DESC

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_sales_per_order
USA,13,1040.49,80.04,7.9
Canada,8,535.59,66.95,7.0
France,5,389.07,77.81,7.8
Brazil,5,427.68,85.54,7.0
Germany,4,334.62,83.66,8.2
United Kingdom,3,245.52,81.84,8.8
Portugal,2,185.13,92.57,6.4
India,2,183.15,91.58,8.7
Czech Republic,2,273.24,136.62,9.1
Other,15,1094.94,73.0,7.4


## Analysis

Although the US has the most total sales, the countries with the highest sales per customer are Czechia, Portugal and India. Canadians are cheap.

# Task 4: Album vs Song Sales

The company would like us to determine whether to continue purchasing full albums to re-sell, or to focus on only the most popular songs in those albums. We want to know how often customers buy entire albums.

To do this, we will check each invoice to see whether it corresponds to an album list or not.

For every invoice we'll need to choose a track, and then create a table with all songs on that track's album, like the one below:

In [14]:
%%sql
SELECT track_id
FROM track t
WHERE album_id = 1

 * sqlite:///chinook.db
Done.


track_id
1
6
7
8
9
10
11
12
13
14


We'll also need table of all tracks on an invoice for each invoice, like this:

In [15]:
%%sql
SELECT track_id
FROM invoice_line il
WHERE il.invoice_id = 1

 * sqlite:///chinook.db
Done.


track_id
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167


We will associate each invoice to an album corresponding to any track, as exemplified below:

In [16]:
%%sql
    SELECT i.invoice_id, MIN(t.album_id) AS album_one
    FROM invoice i
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    INNER JOIN track t on t.track_id = il.track_id
    GROUP BY i.invoice_id
    LIMIT 25

 * sqlite:///chinook.db
Done.


invoice_id,album_one
1,91
2,20
3,203
4,58
5,163
6,5
7,6
8,9
9,20
10,215


Now we will expand on the query above by adding a column that compares whether the album's list of tracks matches the invoice's list of tracks, and save that view.

In [41]:
%%sql
CREATE VIEW invoice_album_bool AS

WITH invoice_and_matching_album AS
        (
        SELECT  i.invoice_id AS invoice_id, 
                MIN(t.album_id) AS album_one
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t on t.track_id = il.track_id
        GROUP BY i.invoice_id
        )

SELECT  invoice_id, 
        CASE 
            WHEN 
                (
                SELECT track_id
                FROM track t
                WHERE t.album_id = ia.album_one

                EXCEPT

                SELECT track_id
                FROM invoice_line il
                WHERE il.invoice_id = ia.invoice_id
                )                    
                IS NULL
             AND
                ( 
                SELECT track_id
                FROM invoice_line il
                WHERE il.invoice_id = ia.invoice_id

                EXCEPT

                SELECT track_id
                FROM track t
                WHERE t.album_id = ia.album_one
                )
                IS NULL
            THEN 'Yes'
            ELSE 'No'
    END AS album_bool
FROM invoice_and_matching_album ia

 * sqlite:///chinook.db
Done.


[]

Let's look at the first 25 invoices and see if this has worked:

In [18]:
%%sql        
SELECT *
FROM invoice_album_bool
LIMIT 25

 * sqlite:///chinook.db
Done.


invoice_id,album_bool
1,Yes
2,No
3,No
4,No
5,Yes
6,No
7,No
8,No
9,No
10,No


Let's check this result by comparing invoice IDs 22 (no album match) and 23 (album match):

In [19]:
%%sql
SELECT invoice_id, track_id
FROM invoice_line il
WHERE (il.invoice_id = 22 OR il.invoice_id = 23)

 * sqlite:///chinook.db
Done.


invoice_id,track_id
22,1000
22,383
23,1
23,6
23,7
23,8
23,9
23,10
23,11
23,12


In [20]:
%%sql
SELECT album_id, track_id
FROM track t
WHERE (album_id = 33 OR album_id = 1)

 * sqlite:///chinook.db
Done.


album_id,track_id
1,1
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14


By comparing the tables we see that invoice 23 does indeed match to an album, and invoice 22 doesn't - hooray! Now we'll add up the total number of invoices corresponding to albums:

In [21]:
%%sql
SELECT  album_bool AS album_status,
        COUNT(album_bool) AS 'Total',
        ROUND(CAST(COUNT(album_bool) AS FLOAT)/(SELECT COUNT(*) FROM invoice_album_bool),2) AS 'Percent'
FROM invoice_album_bool
GROUP BY album_bool

 * sqlite:///chinook.db
Done.


album_status,Total,Percent
No,500,0.81
Yes,114,0.19


## Analysis - Album vs Single

About 20% of users purchase entire albums, so switching to a single-only strategy might mean up to 20% of our consumers go elsewhere if they really prefer buying full albums. However, it's unlikely they would all leave, and the other 80% might buy more singles, so would make up for that lost revenue.

## Further Investigation - Album Buyer Profile

We can further analyze whether our frequent album-buying customers generate significant revenue.

Firstly, we can check how many unique customers bought whole albums.

In [22]:
%%sql
SELECT album_bool, 
        COUNT(DISTINCT(c.customer_id)) AS total,
        ROUND(CAST(COUNT(DISTINCT(c.customer_id)) AS FLOAT) / (SELECT COUNT(DISTINCT(customer_id)) FROM customer) ,2) AS percent
FROM invoice_album_bool ia
INNER JOIN invoice i ON i.invoice_id = ia.invoice_id
INNER JOIN customer c ON c.customer_id = i.customer_id
GROUP BY album_bool

 * sqlite:///chinook.db
Done.


album_bool,total,percent
No,59,1.0
Yes,54,0.92


From the above we can see that almost all customers (54/59) bought at least one album, and all customers bought a single. We can also check how many albums each customer bought:

In [23]:
%%sql
WITH customer_album_count AS
                            (
                            SELECT c.customer_id, COUNT(*) AS albums_purchased
                            FROM invoice_album_bool ia
                            INNER JOIN invoice i ON i.invoice_id = ia.invoice_id
                            INNER JOIN customer c ON c.customer_id = i.customer_id
                            WHERE album_bool = 'Yes'
                            GROUP BY c.customer_id
                            )

SELECT  albums_purchased, 
        COUNT(*) AS number_of_customers,
        ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM customer_album_count),2) AS percent_of_album_buyers
FROM customer_album_count
GROUP BY albums_purchased

 * sqlite:///chinook.db
Done.


albums_purchased,number_of_customers,percent_of_album_buyers
1,20,0.37
2,16,0.3
3,13,0.24
4,3,0.06
5,1,0.02
6,1,0.02


From the above we can see that 2/3 of the 54 customers who purchased an album only purchased 1 or 2 albums, and only 10% purchased 4 or more albums. So we would be most likely to lose those 5 customers who purchased 4+ albums. Customers who purchased 1-2 albums might remain.

## Album Spending vs Total Spending
We can further profile those who spend on albums by comparing their total spending to their album spending. First we'll create a table of customer's album vs total spending and then focus on customers who have above-average total spending, since we don't want to lose those customers.

In [43]:
%%sql
CREATE VIEW customer_album_total_spending AS

WITH invoice_customer_total_album AS
        (
        SELECT  i.invoice_id AS invoice_id,
                c.customer_id AS customer_id,
                i.total AS total, 
                ia.album_bool AS album_bool
        FROM invoice_album_bool ia
        INNER JOIN invoice i ON i.invoice_id = ia.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id
        ),
     customer_album_spending AS
        (
        SELECT customer_id, SUM(total) AS album_spending
        FROM invoice_customer_total_album
        WHERE album_bool = 'Yes'
        GROUP BY customer_id
        )

SELECT  icta.customer_id AS customer_ID,
        ROUND(COALESCE(cas.album_spending,0),2) AS album_spending,
        ROUND(SUM(total),2) AS total_spending,
        ROUND(COALESCE(cas.album_spending,0)/ SUM(total),2) AS percent
FROM invoice_customer_total_album icta
LEFT JOIN customer_album_spending cas ON cas.customer_id = icta.customer_id 
GROUP BY icta.customer_id, album_spending

 * sqlite:///chinook.db
Done.


[]

Below are the customers whose purchases were at least 50% albums:

In [44]:
%%sql
SELECT *
FROM customer_album_total_spending cats
ORDER BY percent DESC
LIMIT 8

 * sqlite:///chinook.db
Done.


customer_ID,album_spending,total_spending,percent
37,73.26,94.05,0.78
3,61.38,99.99,0.61
40,39.6,72.27,0.55
52,37.62,68.31,0.55
55,43.56,81.18,0.54
25,39.6,76.23,0.52
47,25.74,50.49,0.51
51,37.62,75.24,0.5


In [26]:
%%sql
SELECT AVG(percent)
FROM customer_album_total_spending

 * sqlite:///chinook.db
Done.


AVG(percent)
0.30440677966101704


From the above we see that 8 of 59 customers (or 14%) spend at least 50% of their purchases on albums. This means purchasing albums is a feature they likely depend on. The average percentage is 30%, as calculated above. 

Are these customers big spenders, too? Based on the above, we'll categorize customers by the percent of their purchases spent on albums as follows:

Category | % Spent on Albums
---- | :----
High | 50-100%
Medium | 30-50%
Low | 0-30%

The table below shows the categories of all customers who spent more than the average total spending:

In [27]:
%%sql
SELECT  customer_id, 
        total_spending,
        CASE
            WHEN percent >= 0.5 THEN 'High'
            WHEN percent >= 0.30 THEN 'Medium'
            ELSE 'Low' END as album_spending_category
FROM customer_album_total_spending cats
WHERE total_spending > (SELECT AVG(total_spending) FROM customer_album_total_spending)
ORDER BY total_spending DESC

 * sqlite:///chinook.db
Done.


customer_ID,total_spending,album_spending_category
5,144.54,Medium
6,128.7,Low
46,114.84,Medium
58,111.87,Low
1,108.9,Medium
13,106.92,Medium
34,102.96,Medium
3,99.99,High
42,99.99,Low
17,98.01,Low


There are only three above-average spenders who also purchase a high amount of albums. Below we'll find the average purchase of high, medium, and low album spenders.

In [28]:
%%sql
SELECT CASE
            WHEN percent >= 0.5 THEN 'High'
            WHEN percent >= 0.30 THEN 'Medium'
            ELSE 'Low' END as album_spending_category,
        ROUND(AVG(total_spending),2) as average_spending
FROM customer_album_total_spending cats
WHERE total_spending > (SELECT AVG(total_spending) FROM customer_album_total_spending)
GROUP BY album_spending_category
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


album_spending_category,average_spending
Medium,102.56
Low,96.28
High,91.74


Among above-average spenders, those who buy 30-50% albums spend the most, on average. Those who buy over 50% albums spent about \\$10 less on average, which as the category of above-average spenders has a range of about $65, is significant. The range of values is calculated below:

In [29]:
%%sql
SELECT AVG(total_spending), MAX(total_spending), MAX(total_spending) - AVG(total_spending) as range
FROM customer_album_total_spending

 * sqlite:///chinook.db
Done.


AVG(total_spending),MAX(total_spending),range
79.82084745762712,144.54,64.71915254237287


We see similar results for all customers in the table below: high album-spenders do not have the highest average spending, though the difference is less pronounced than among above-average spenders.

In [30]:
%%sql
SELECT CASE
            WHEN percent >= 0.5 THEN 'High'
            WHEN percent >= 0.30 THEN 'Medium'
            ELSE 'Low' END as album_spending_category,
        ROUND(AVG(total_spending),2) as average_spending
FROM customer_album_total_spending cats
GROUP BY album_spending_category
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


album_spending_category,average_spending
Medium,82.91
Low,77.84
High,77.22


Lastly, we can identify the high album-spending, high total-spending customers.

In [31]:
%%sql
SELECT c.first_name || " " || c.last_name AS customer_name, cats.total_spending, cats.percent AS percent_albums
FROM customer_album_total_spending cats
INNER JOIN customer c ON c.customer_id = cats.customer_id
WHERE (total_spending > (SELECT AVG(total_spending) FROM customer_album_total_spending)) AND (percent>0.5)

 * sqlite:///chinook.db
Done.


customer_name,total_spending,percent_albums
François Tremblay,99.99,0.61
Fynn Zimmermann,94.05,0.78
Mark Taylor,81.18,0.54


So of 59 customers, these are the 3 that we would be most concerns about losing as customers. We might extend promotions to these customers to retain them.

Overall, the evidence suggests that a considerable percentage of current customers use the album-buying feature, though those that use it the most do not necessarily spend the most. Changing to a single-track-purchase strategy would risk losing these customers, but those who buy only 1-2 albums might also stick with this platform since they're already invested. Further research should be done on those who are in this middle zone.

In addition, switching to single tracks only might also attract customers who spend more money on popular singles, which might offest the loss of album-buyers. Further investigation could be done to determine which types of single tracks are most popular.